<p><img alt="Colaboratory logo" height="99px" 
src='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTpw-4GbEpC5PiordgozbjH7ILPI-X6NU_d7Q&usqp=CAU'
align="left" hspace="10px" vspace="0px" ></p>
<h1>Prever Vendas Futuras 
<p>Projeto final para o curso Coursera "Como ganhar um concurso de ciência de dados"</p>
</h1>
<hr />
<a href="https://www.kaggle.com/c/competitive-data-science-predict-future-sales/rules">Kaggle</a>


##### Descrição
Este desafio serve como projeto final para o curso Coursera "Como ganhar um concurso de ciência de dados" .

Nesta competição, você trabalhará com um conjunto de dados de série temporal desafiador que consiste em dados diários de vendas, gentilmente cedidos por uma das **maiores empresas de software da russa** - a **1C Company** . 

Pedimos que você preveja as vendas totais de cada produto e loja no próximo mês. Ao resolver esta competição, você poderá aplicar e aprimorar suas habilidades em ciência de dados.


<p><img alt="Colaboratory logo" height="49px" 
src='https://static.1c.ru/images/logo.png'
align="left" hspace="10px" vspace="0px" ></p>
<h1> 1C COMPANY
</h1>
Fundada em 1991, a 1C Company é especializada em desenvolvimento, distribuição, publicação e suporte de software para o mercado de massa. 
 <a href="https://1c.ru/eng/title.htm">site daa empresa</a>

##### Dados

Você recebe dados históricos de vendas diários. A tarefa é prever a quantidade total de produtos vendidos em cada loja para o conjunto de teste. Observe que a lista de lojas e produtos muda ligeiramente a cada mês. Criar um modelo robusto que possa lidar com tais situações é parte do desafio.

**Descrições de arquivo**
> * **sales_train.csv** - o conjunto de treinamento. Dados históricos diários de janeiro de 2013 a outubro de 2015.
> * **test.csv** - o conjunto de teste. Você precisa prever as vendas dessas lojas e produtos para novembro de 2015.
> * **sample_submission.csv** - um arquivo de envio de amostra no formato correto.
> * **items.csv** - informações complementares sobre os itens / produtos.
> * **item_categories.csv**  - informações complementares sobre as categorias de itens.
> * **shops.csv** - informações complementares sobre as lojas.

**Campos de dados**
> * **ID  - um Id que representa uma tupla (Loja, Item) dentro do conjunto de teste
> * **shop_id** - identificador único de uma loja
> * **item_id** - identificador único de um produto
> * **item_category_id** - identificador único da categoria do item
> * **item_cnt_day** - número de produtos vendidos. **Você está prevendo um valor mensal desta medida**  💥
> * **item_price** - preço atual de um item
> * **data**  - data no formato dd / mm / aaaa
> * **date_block_num** - um número de mês consecutivo, usado por conveniência. Janeiro de 2013 é 0, fevereiro de 2013 é 1, ..., outubro de 2015 é 33
> * **item_name**  - nome do item
> * **shop_name** - nome da loja
> * **item_category_name** - nome da categoria do item

##### Regras

>* **Limites de envio**: No máximo 5 inscrições por dia.

[Submissions](https://www.kaggle.com/c/competitive-data-science-predict-future-sales/submissions)

#### Analises

##### Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
class Output:
    def clear(self):
        clear_output()
output = Output()
sns.set(rc={"figure.figsize": (10, 2)})
import gc
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:

def plot(d,t='area'):
  if t =='area':
    return pd.DataFrame(d).sort_index().plot.area()
  if t =='line':
    return pd.DataFrame(d).sort_index().plot.line()
  if t =='hist':
    return pd.DataFrame(d).sort_index().plot.hist()
  if t =='bar':
    return pd.DataFrame(d).sort_index().plot.bar()

def transpose(d,l=100):
  x = pd.DataFrame(d)
  return x.head(l).transpose()

In [ ]:
_path = '/kaggle/input/competitive-data-science-predict-future-sales/'
item_categories =  pd.read_csv(_path+'/item_categories.csv')
items =  pd.read_csv(_path+'/items.csv')
sales_train =  pd.read_csv(_path+'/sales_train.csv')
submission =  pd.read_csv(_path+'/sample_submission.csv')
shops =  pd.read_csv(_path+'/shops.csv')
test =  pd.read_csv(_path+'/test.csv')

##### Traduzindo dados...

In [ ]:
!pip install -U textblob
from textblob import TextBlob
for i in range(len(shops)):
  text = shops.loc[i,"shop_name"]
  hi_blob = TextBlob(text)
  text_pt = hi_blob.translate(to='pt',from_lang='ru')
  shops.loc[i,"shop_name"] =str(text_pt)
output.clear()

In [ ]:
for i in range(len(item_categories)):
  text = item_categories.loc[i,"item_category_name"]
  hi_blob = TextBlob(u''+text)
  text_pt = hi_blob.translate(to='pt')
  item_categories.loc[i,"item_category_name"] =str(text_pt)
output.clear()

###### Overview

> **shops**

In [ ]:
shops.describe().transpose()

In [ ]:
shops.shop_id.plot.line()
plt.show()

*Obtendo mais informações da wikipedia...*

In [ ]:
import urllib.request
import codecs
from bs4 import BeautifulSoup
wiki = 'https://pt.wikipedia.org/wiki/Lista_de_cidades_da_R%C3%BAssia_por_popula%C3%A7%C3%A3o'
page = urllib.request.urlopen(wiki)
soup = BeautifulSoup(page, 'html5lib')
table = soup.find('table', attrs={'class': 'wikitable'})
cities = []
tds = [row for row in table.findAll('tr')]
for td in tds:
  if len(td.findAll("td"))==4:
    cities.append([td.findAll("td")[1].find("a").text.strip().upper(),
                 td.findAll("td")[2].find("a").text.strip().upper(),
                 td.findAll("td")[3].find("i").text.replace('\xa0','').strip()])
cities


In [ ]:
!pip install fuzzywuzzy
!pip install python-Levenshtein
output.clear()

In [ ]:
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 
import re
def fn_city(n):
  n if n.find('"')==-1 else n[0:n.find('"')].strip()
  n = re.sub('[^A-Za-z]+', ' ', n).strip().upper()
  n = n.replace("CENTRO","")
  n = n.replace("CENTRAL","")
  n = n.replace("COMERCIAL","")
  n = n.replace("COMERCIO","")
  n = n.replace("CENTRAL","")
  n = n.replace("N ","")
  n = n.replace("DE ","")
  n = n.replace("MEGA ","")
  n = n.replace("COMPLEXO E ","")
  if n.find("SPB")>=0:
    n = "Distrito Federal do Noroeste"
  else:
    if n.find("UFA")>=0:
      n = "Bascortostão"
    else:
      if n.find("MOSC")>=0:
        n = "distrito federal central"
      else:
        n = n.split(' ')[0]
        c = [m[1] for m in cities]
        n, s= process.extractBests(n, c)[0]  
  return n
shops["shop_shortname"] = shops.shop_name.transform(lambda n:fn_city(n))

shops["shop_city"] = ""
shops["shop_district"]=""
shops["shop_population"]=0

query = list(shops.shop_shortname.value_counts().index)
output.clear()
for q in query:
  print(q.lower())
  r, i = process.extractBests(q, cities)[0]
  shops.loc[(shops.shop_shortname==q),"shop_city"] = r[0]
  shops.loc[(shops.shop_shortname==q),"shop_district"]= r[1]
  shops.loc[(shops.shop_shortname==q),"shop_population"]= int(r[2])

In [ ]:
transpose(shops.shop_city.sort_values())

In [ ]:
shops["shop_category"] = shops.shop_name.str.split(" ").map( lambda x: x[1].replace("(","").replace(",","").replace(".","").replace('\"','').upper()[0:1] )
cats = list(shops.shop_category.value_counts().index)
shops.loc[(shops.shop_category.isin(cats[3:])),"shop_category"]='O'
transpose(shops.shop_category.value_counts())

In [ ]:
cols = [ 'shop_id', 'shop_city', 'shop_district',  'shop_category']
shops = shops[cols]
shops.shop_district = shops.shop_district.astype('category')
shops.shop_district = shops.shop_district.cat.codes
shops.shop_category = shops.shop_category.astype('category')
shops.shop_category = shops.shop_category.cat.codes
shops.shop_city = shops.shop_city.astype('category')
shops.shop_city = shops.shop_city.cat.codes
shops.head()

In [ ]:
shops.shop_city.sort_index().plot.hist()
plt.show()

**item_categories**

In [ ]:
item_categories.head(1)

In [ ]:
item_categories["type"] = item_categories.item_category_name.apply( lambda x: x.split(" ")[0] ).astype(str)

In [ ]:
item_categories["type"].value_counts()

In [ ]:
from subprocess import check_output
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)
wordcloud = WordCloud(
                          background_color='white',
                          stopwords=stopwords,
                          max_words=120,
                          max_font_size=410, 
                          random_state=42
                         ).generate(str(item_categories["type"]))

fig = plt.figure(1)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
item_categories.describe().transpose()

In [ ]:
item_categories["item_category_type"] =  item_categories.item_category_name+'- '
item_categories["item_category_type"] = item_categories.item_category_type.transform(lambda c: c.split("-")[0].strip().upper() if c.find(" ")==-1 else c.split(" ")[0].strip().upper()  )
item_categories["item_category_type"] = item_categories.item_category_type.transform(lambda c: c if c.find("-")==-1 else c.split("-")[0].strip().upper()  )
categories = (item_categories["item_category_type"].value_counts().index)
item_categories.loc[(item_categories.item_category_type.isin(categories[8:])),"item_category_type"]='OUTROS'
transpose(item_categories.item_category_type.value_counts()).transpose()

**items**

In [ ]:
import re
def name(x):
    x = x.lower() 
    x = x.partition('[')[0]
    x = x.partition('(')[0] 
    x = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', x) 
    x = x.replace('  ', ' ') 
    x = x.strip() 
    hi_blob = TextBlob(x)
    x = hi_blob.translate(to='pt',from_lang='ru')
    return str(x).upper()

sample = items.sample(5)
sample["item_name"] = sample["item_name"].apply(lambda x: name(x))
sample.head(5)

In [ ]:
items.describe().transpose()

In [ ]:
items.item_category_id.sort_index().plot.hist()
plt.show()

**items_categories**

In [ ]:
items_categories = items.merge(item_categories,how='left',left_on='item_category_id', right_on='item_category_id')

In [ ]:
items_categories.describe().transpose()

In [ ]:
items_categories = items_categories[["item_id","item_category_id","item_category_type"]]
items_categories.reset_index(drop=True, inplace=True)
items_categories.head()

In [ ]:
items_categories.item_category_id.sort_index().plot.hist()
plt.show()

**sales_train**

In [ ]:
sales_train.head()

In [ ]:
sales_train.describe().transpose()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
sales_train.item_price.sort_index().plot.hist(ax=ax1)
ax1.set_xlabel("item_price")
sales_train.item_cnt_day.sort_index().plot.hist(ax=ax2)
ax2.set_xlabel("item_cnt_day")
plt.show()

**sales**

In [ ]:
sales = sales_train.copy()
sales = items.merge(sales,how='right',left_on='item_id', right_on='item_id')
len(sales),len(sales_train),len(items)

In [ ]:
sales.head()

In [ ]:
sales.describe()

In [ ]:
sales = items_categories.merge(sales,how='right',on=["item_id","item_category_id"])
len(sales),len(sales_train),len(items_categories)

In [ ]:
sales.date_block_num.sort_index().plot.hist()

In [ ]:
import time
ts = time.time()
sales.date  = pd.to_datetime(sales.date,errors='coerce')
years = (sales.date.dt.year.value_counts().index)
months= (sales.date.dt.month.value_counts().index)
years
sequence = 0
for i,y in enumerate(years.sort_values()):
  for m in months.sort_values():
    if y==2013 and m ==1:
      sales.loc[(sales.date.dt.year==y)&(sales.date.dt.month==m),"date_block_num"]=0
    else:
      sequence+=1
      sales.loc[(sales.date.dt.year==y)&(sales.date.dt.month==m),"date_block_num"]=sequence
gc.collect()
time.time() - ts

In [ ]:
sales["dayofweek"]= sales.date.dt.dayofweek
sales["weekday"]= sales.date.dt.weekday
sales["quarter"]= sales.date.dt.quarter
sales["week"]= sales.date.dt.week
sales["month"]= sales.date.dt.month
#sales["year"]= sales.date.dt.year
sales["weekofyear"]= sales.date.dt.weekofyear
sales.head()

In [ ]:
sales.date_block_num.sort_index().plot.hist()

In [ ]:
transpose(sales.item_price.describe())

In [ ]:
transpose(sales.item_price.describe())

In [ ]:
transpose(sales.item_cnt_day.describe())

In [ ]:
transpose(sales.item_cnt_day.describe())

###### **Pivot Table**

In [ ]:
sales.pivot_table(index=["item_cnt_day"])

In [ ]:
sales.pivot_table(index=["shop_id"],columns=["date_block_num"], aggfunc=["mean"])

In [ ]:
pd.pivot_table(sales,index=["date_block_num","shop_id","item_category_id"],values=["item_cnt_day"],aggfunc=["sum","mean"])

In [ ]:
pd.pivot_table(sales,index=["date_block_num","shop_id","item_category_id","item_id"],values=["item_cnt_day"],aggfunc=["sum","mean"])

In [ ]:
pd.pivot_table(sales,index=["date_block_num","shop_id","item_category_id","item_id"],values=["item_cnt_day"],aggfunc=["sum","mean"])

In [ ]:
sales.head()

In [ ]:
pd.pivot_table(sales,index=["date_block_num","month"],values=["item_cnt_day"],aggfunc=["sum"])

In [ ]:
pd.pivot_table(sales,index=["shop_id"],values=["item_cnt_day"],aggfunc=['mean','max','min','count'])